In [2]:
from tqdm import tqdm
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

In [3]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(28 * 28, 200)
        self.fc2 = nn.Linear(200, 10)

    def forward(self, x):
        x = x.view((-1, 28 * 28))
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x


In [4]:
class Normalize(nn.Module):
    def forward(self, x):
        return (x - 0.1307) / 0.3081


In [5]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

batch_size = 512
seed = 42
learning_rate = 0.01
num_epochs = 10
eps = 0.1
k = 7
trades_lambda = 1.0

# Setting the random number generator
torch.manual_seed(seed)

In [6]:
# Datasets
train_dataset = datasets.MNIST('mnist_data/', train=True, download=True, transform=transforms.Compose([transforms.ToTensor()]))
test_dataset = datasets.MNIST('mnist_data/', train=False, download=True, transform=transforms.Compose([transforms.ToTensor()]))

# Data loaders
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|████████████████████████████████████████████████████████████████████| 9912422/9912422 [00:37<00:00, 263916.00it/s]


Extracting mnist_data/MNIST\raw\train-images-idx3-ubyte.gz to mnist_data/MNIST\raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|████████████████████████████████████████████████████████████████████████| 28881/28881 [00:00<00:00, 245411.18it/s]


Extracting mnist_data/MNIST\raw\train-labels-idx1-ubyte.gz to mnist_data/MNIST\raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|███████████████████████████████████████████████████████████████████| 1648877/1648877 [00:00<00:00, 1862190.71it/s]


Extracting mnist_data/MNIST\raw\t10k-images-idx3-ubyte.gz to mnist_data/MNIST\raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|█████████████████████████████████████████████████████████████████████████| 4542/4542 [00:00<00:00, 1518454.39it/s]

Extracting mnist_data/MNIST\raw\t10k-labels-idx1-ubyte.gz to mnist_data/MNIST\raw



In [7]:
# Add data normalization as a first "layer" to the network
# This allows us to search for adversarial examples to the real image,
# rather than to the normalized image
model = nn.Sequential(Normalize(), Net())
model = model.to(device)

opt = optim.Adam(params=model.parameters(), lr=learning_rate, weight_decay=1e-5)
scheduler = optim.lr_scheduler.StepLR(opt, 15)
ce_loss = torch.nn.CrossEntropyLoss()
kl_loss = torch.nn.KLDivLoss(reduction='batchmean')

In [8]:
def pgd(model, x_batch, target, k, eps, eps_step, kl_loss: bool = False):
    if kl_loss:
        # Loss function for the case that target is a distribution rather than a label (used for TRADES)
        loss_fn = torch.nn.KLDivLoss(reduction='sum')
    else:
        # Standard PGD
        loss_fn = torch.nn.CrossEntropyLoss(reduction='sum')
    
    # Disable gradients here
    with torch.no_grad():
        # Initialize with a random point inside the considered perturbation region
        x_adv = x_batch.detach() + eps * (2 * torch.rand_like(x_batch) - 1)
        
       # Project back to the image domain
        x_adv.clamp(min=0.0, max=1.0)

        for step in range(k):
            # Make sure we don't have a previous compute graph and enable gradient computation
            x_adv.detach_().requires_grad_()

            # Re-enable gradients
            with torch.enable_grad():
                # Run the model and obtain the loss
                out = F.log_softmax(model(x_adv), dim=1)
                model.zero_grad()

                # Compute gradient
                loss_fn(out, target).backward()
            
            # Compute step
            step = eps_step * x_adv.grad.sign()

            # Project to eps ball
            x_adv = x_batch + (x_adv + step - x_batch).clamp(min=-eps, max=eps)

            # Clamp back to image domain: we clamp at each step
            x_adv.clamp_(min=0.0, max=1.0)
    
    return x_adv.detach()

In [9]:
def train_and_test_accuracies_using_defense(defense, num_epochs, train_loader, test_loader, k, eps):
    for epoch in range(1, num_epochs + 1):
        # Training
        for _, (x_batch, y_batch) in enumerate(tqdm(train_loader)):

            x_batch, y_batch = x_batch.to(device), y_batch.to(device)
            
            if defense == 'PGD':
                # PGD attack to generate adversarial examples
                
                # Switch model to eval mode, to ensure it is deterministic
                model.eval()

                x_adv = pgd(
                    model, 
                    x_batch=x_batch, 
                    target=y_batch,
                    eps=eps, 
                    k=k, 
                    eps_step = 2.5 * eps / k
                )

                # Switch back to training mode
                model.train()
                out_pgd = model(x_adv)

                # Compute loss
                loss = ce_loss(out_pgd, y_batch)

            elif defense == 'TRADES':
                # Switch to training mode
                model.train()
                out_nat = model(x_batch)
                target = F.softmax(out_nat.detach(), dim=1)

                # Do PGD attack to generate adversarial examples
                
                # Switch network to eval mode, to ensure it is deterministic
                model.eval()

                x_adv = pgd(
                    model, 
                    x_batch=x_batch, 
                    target=target, 
                    k=k, 
                    eps=eps,
                    eps_step=2.5 * eps / k,
                    kl_loss=True
                )

                # Calculate loss
                
                # Switch to training mode
                model.train()
                out_adv = F.log_softmax(model(x_adv), dim=1)
                
                loss_nat = ce_loss(out_nat, y_batch)
                loss_adv = kl_loss(out_adv, target)
                loss = loss_nat + trades_lambda * loss_adv
                
            elif defense == 'none':                
                model.train()
                out_nat = model(x_batch)
                loss = ce_loss(out_nat, y_batch)

            opt.zero_grad()
            loss.backward()
            opt.step()

        # Testing
        model.eval()

        tot_test, tot_acc, tot_adv_acc = 0.0, 0.0, 0.0

        for _, (x_batch, y_batch) in enumerate(tqdm(test_loader)):
            x_batch, y_batch = x_batch.to(device), y_batch.to(device)

            # Prediction by the model on each batch
            out = model(x_batch)
            pred = torch.max(out, dim=1)[1]
            acc = pred.eq(y_batch).sum().item()

            x_adv = pgd(
                model,
                x_batch=x_batch,
                target=y_batch,
                k=k,
                eps=eps,
                eps_step=2.5 * eps / k
            )

            # Prediction of the model on the adversarial batch
            out_adv = model(x_adv)
            pred_adv = torch.max(out_adv, dim=1)[1]
            acc_adv = pred_adv.eq(y_batch).sum().item()

            # Add to total accuracies for both regular and adversarial accuracies
            tot_acc += acc
            tot_adv_acc += acc_adv
            tot_test += x_batch.size()[0]

        scheduler.step()

        print('Epoch %d: Accuracy %.5lf, Adv Accuracy %.5lf' %
            (epoch, tot_acc / tot_test, tot_adv_acc / tot_test))


In [10]:
# Evaluate model using standard training, no defense
train_and_test_accuracies_using_defense(
    defense='none', num_epochs=num_epochs, train_loader=train_loader, test_loader=test_loader, k=k, eps=eps)

100%|██████████████████████████████████████████████████████████████████████████████████| 20/20 [00:04<00:00,  4.96it/s]


Epoch 1: Accuracy 0.95870, Adv Accuracy 0.28760


100%|██████████████████████████████████████████████████████████████████████████████████| 20/20 [00:04<00:00,  4.80it/s]


Epoch 2: Accuracy 0.96450, Adv Accuracy 0.30990


100%|██████████████████████████████████████████████████████████████████████████████████| 20/20 [00:03<00:00,  5.01it/s]


...

Epoch 8: Accuracy 0.96850, Adv Accuracy 0.24830


100%|██████████████████████████████████████████████████████████████████████████████████| 20/20 [00:04<00:00,  4.77it/s]


Epoch 9: Accuracy 0.97310, Adv Accuracy 0.23370


100%|██████████████████████████████████████████████████████████████████████████████████| 20/20 [00:04<00:00,  4.82it/s]

Epoch 10: Accuracy 0.97220, Adv Accuracy 0.26970


In [11]:
# Evaluate model using PGD defense
train_and_test_accuracies_using_defense(
    defense='PGD', num_epochs=num_epochs, train_loader=train_loader, test_loader=test_loader, k=k, eps=eps)

100%|██████████████████████████████████████████████████████████████████████████████████| 20/20 [00:04<00:00,  4.82it/s]


Epoch 1: Accuracy 0.96460, Adv Accuracy 0.79940


100%|██████████████████████████████████████████████████████████████████████████████████| 20/20 [00:04<00:00,  4.97it/s]


Epoch 2: Accuracy 0.97100, Adv Accuracy 0.83390


100%|██████████████████████████████████████████████████████████████████████████████████| 20/20 [00:04<00:00,  4.76it/s]


...

Epoch 8: Accuracy 0.98180, Adv Accuracy 0.88980


100%|██████████████████████████████████████████████████████████████████████████████████| 20/20 [00:04<00:00,  4.70it/s]


Epoch 9: Accuracy 0.98190, Adv Accuracy 0.88870


100%|██████████████████████████████████████████████████████████████████████████████████| 20/20 [00:04<00:00,  4.76it/s]

Epoch 10: Accuracy 0.98240, Adv Accuracy 0.88900


In [12]:

# Evaluate model using PGD defense
train_and_test_accuracies_using_defense(
    defense='TRADES', num_epochs=num_epochs, train_loader=train_loader, test_loader=test_loader, k=k, eps=eps)


100%|██████████████████████████████████████████████████████████████████████████████████| 20/20 [00:04<00:00,  4.81it/s]


Epoch 1: Accuracy 0.98230, Adv Accuracy 0.88770


100%|██████████████████████████████████████████████████████████████████████████████████| 20/20 [00:03<00:00,  5.13it/s]


Epoch 2: Accuracy 0.98330, Adv Accuracy 0.88950


100%|██████████████████████████████████████████████████████████████████████████████████| 20/20 [00:03<00:00,  5.02it/s]


...

Epoch 8: Accuracy 0.98460, Adv Accuracy 0.88860


100%|██████████████████████████████████████████████████████████████████████████████████| 20/20 [00:04<00:00,  4.93it/s]


Epoch 9: Accuracy 0.98410, Adv Accuracy 0.88880


100%|██████████████████████████████████████████████████████████████████████████████████| 20/20 [00:04<00:00,  4.83it/s]

Epoch 10: Accuracy 0.98340, Adv Accuracy 0.89100
